<a href="https://colab.research.google.com/github/dtabuena/Workshop/blob/main/Ephys_v2/Analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
'Get Standard Modules'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from scipy.signal import butter,filtfilt
from tqdm import tqdm
import warnings
from IPython.display import clear_output
from datetime import datetime
import sys
import warnings
import shutil
from google.colab import files
!pip install openpyxl --quiet
!pip install XlsxWriter --quiet


'''Get Repositories'''
try: shutil.rmtree('/content/EphysLib')
except: None

"run dtabuena's ephys notebooks"
!git clone https://github.com/dtabuena/EphysLib
to_import = [
            'ABF_Quality_Control.ipynb',
            'Basic_Ephys.ipynb',
            'Simple_ABF_tools.ipynb',
            'fun_math.ipynb',
            'importing_abfs_from_dropbox.ipynb',
            'QC_recoding_dataframe.ipynb',
            'Analyzers/input_resistance_analyzer.ipynb',
            'Analyzers/gain_analyzer.ipynb',
            'Analyzers/latencey_analyzer.ipynb',
            'Analyzers/IV_analyzer.ipynb',
            'Analyzers/Vm_analyzer.ipynb',
            'Analyzers/membrane_analyzer.ipynb',
            'Analyzers/rheobase_analyzer.ipynb',

            'Ephys_wrapper.ipynb',
            ]
for i in to_import:
    f = '/content/EphysLib/' + i
    %run $f --quiet


##### Setup Protocol List
VC_prot = ['VC - MemTest-10ms-160ms',
           'VC - Multi IV - 150ms',]
IC_prot = ['IC - Gain - D20pA',
           'IC - Gain - D50pA',
           'IC - Rheobase',
           'IC - R input',
           'IC - Latentcy 800pA-1s'
           'VC - 3min GapFree',
           'I0 - 3min GapFree']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 3.5 MB/s eta 0:00:00
Cloning into 'EphysLib'...
remote: Enumerating objects: 1141, done.
remote: Counting objects: 100% (518/518), done.
remote: Compressing objects: 100% (247/247), done.
remote: Total 1141 (delta 377), reused 359 (delta 271), pack-reused 623
Receiving objects: 100% (1141/1141), 17.41 MiB | 15.96 MiB/s, done.
Resolving deltas: 100% (754/754), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.1 MB/s eta 0:00:00


In [1]:
import logging
def analyze_dataset( dataset,valid_protocols, log_level,spike_args,manual_exclusions=None):
    ''' Analyze are recordings in a dataset '''
    results_dict={}

    ''' Get Dataset '''
    file_loc = get_drobox_folder(dataset['data_source'],  dataset['data_name'])
    abf_recordings_df, protocol_set = catalogue_recs(file_loc,dataset['file_naming_scheme'])
    results_dict['abf_recordings_df'] = abf_recordings_df
    results_dict['protocol_set'] = protocol_set
    abf_recordings_df, _ = purge_wrong_clamp(abf_recordings_df,VC_prot,IC_prot)




In [ ]:
dataset = {'data_name': 'Wild_Type_Hipp',
           'data_source': "https://www.dropbox.com/scl/fo/xpsvbsuu6nzsl4rtm7fvv/h?rlkey=wj4gdwxalt4wse40um2z2fvjx&dl=1",
           'file_naming_scheme': ['Rec_date','GenoType','Sex','Age','SliceDir','Slice_Num','Cell_num','Cell_Type'],
           }




In [6]:
import json
import urllib
urllib.request.urlretrieve( 'https://github.com/dtabuena/Workshop/raw/1b79dca6d197f411442a908ea00ac5a7267e8247/Ephys_v2/analysis_args.json', "./analysis_args.json")
with open('./analysis_args.json', 'r') as my_json:
    analysis_args = json.load(my_json)

In [ ]:
def analyze_rmp(abf,analysis_args):
    pass
def analyze_rheobase(abf,analysis_args):
    pass
def analyze_gain(abf,analysis_args):
    pass
def analyze_membrane_props(abf,analysis_args):
    pass
def analyze_latencey(abf,analysis_args):
    pass
def analyze_R_input(abf,analysis_args):
    pass
def analyze_IV_curves(abf,analysis_args):
    pass

In [ ]:
def get_drobox_folder(link, new_filename):
    'Download a folder from dropbox and unzip'
    zipped_file_path = "/content/"+new_filename + ".zip"
    unzipped_file_path = "/content/"+new_filename
    if not( os.path.exists(zipped_file_path)):
        !wget -O $zipped_file_path $link    # download with new name
    !echo A | unzip $zipped_file_path -d $unzipped_file_path
    return new_filename

In [ ]:
def ephys_wrapper(dataset,VC_prot,IC_prot,strat_cols=['Cell_Type'],verbose=False, spike_args={'spike_thresh':20, 'high_dv_thresh': 50,'low_dv_thresh': -30,'window_ms': 2},manual_exclusions=[]):
    '''wrapper for single dataset pipeline'''
    results = {}
    try:
        '''Unpack'''
        data_name = dataset['data_name']
        data_source = dataset['data_source']
        file_naming_scheme = dataset['file_naming_scheme']



        ''' Gather and Catalog Source Data'''
        file_loc = get_drobox_folder(data_source, data_name)
        # clear_output(wait=False)
        abf_recordings_df, protocol_set = catalogue_recs(file_loc,file_naming_scheme)
        results['abf_recordings_df'] = abf_recordings_df
        results['protocol_set'] = protocol_set


        abf_recordings_df, _ = purge_wrong_clamp(abf_recordings_df,VC_prot,IC_prot)

        results['abf_recordings_df'] = abf_recordings_df

        csv_name = cell_prot_lut(abf_recordings_df,protocol_set,csv_name=data_name+'_Recording_LookUp')
        results['prot_lut'] = csv_name

        '''Set Internal Analysis Params'''
        print(spike_args)

        func_dict = {}
        arg_dict = {}

        func_dict['VC - 3min GapFree']= rmp_analyzer
        arg_dict['VC - 3min GapFree'] = [True] # [to_plot?]

        func_dict['I0 - 3min GapFree']= rmp_analyzer
        arg_dict['I0 - 3min GapFree'] = [True] # [to_plot?]

        func_dict['IC - Rheobase']= rheobase_analyzer
        arg_dict['IC - Rheobase'] = [spike_args, True, False, False]  # [spike_args, to_plot, verbose, force_singlespike]

        func_dict['IC - Gain - D10pA']= gain_analyzer
        arg_dict['IC - Gain - D10pA']= [spike_args, 1, 4, .7,[-60,-80]]  # [spike_args, to_plot [0:2], max_fit_steps, rel_slope_cut, Vh_hilo]

        func_dict['IC - Gain - D20pA']= gain_analyzer
        arg_dict['IC - Gain - D20pA']= [spike_args, 1, 4, .7,[-60,-80]]  # [spike_args, to_plot [0:2], max_fit_steps, rel_slope_cut, Vh_hilo]

        func_dict['IC - Gain - D25pA']= gain_analyzer
        arg_dict['IC - Gain - D25pA']= [spike_args, 1, 4, .7,[-60,-80]]  # [spike_args, to_plot [0:2], max_fit_steps, rel_slope_cut, Vh_hilo]

        func_dict['IC - Gain - D50pA']= gain_analyzer
        arg_dict['IC - Gain - D50pA']= arg_dict['IC - Gain - D20pA']

        func_dict['VC - MemTest-10ms-160ms']= membrane_analyzer
        arg_dict['VC - MemTest-10ms-160ms']= [True, False, ['Ra', 'Rm', 'Cm', 'tau',	'Cmq',	'Cmf',	'Cmqf', 'Cm_pc']]  # [to_plot, verbose]

        func_dict['IC - Latentcy 800pA-1s']= latencey_analyzer
        arg_dict['IC - Latentcy 800pA-1s']= [spike_args, True]  # [spike_args, to_plot]

        func_dict['IC - R input']= input_resistance_analyzer
        arg_dict['IC - R input']= [[-30, 10] ,True]  # [dVm_limits, to_plot]

        func_dict['VC - Multi IV - 150ms'] = IV_analyzer_v2
        arg_dict['VC - Multi IV - 150ms']= [{'IV_Early':(16.5, 30),'IV_Steady_State':(100,120)} ,[False, True]]  # [measure_windows, to_plot]


        '''Analyze Dataset'''
        abf_recordings_df, problem_recs = analysis_iterator(abf_recordings_df,func_dict,arg_dict,verbose=verbose)
        # clear_output(wait=True)
        print('problem_recs')
        _=[print('     '+r) for r in problem_recs]
        results['problem_recs'] = problem_recs
        results['abf_recordings_df'] = abf_recordings_df

        '''Download Analysis figs'''
        zip_name = '/content/' + data_name + '_Saved_Figs.zip'
        !zip -r $zip_name /content/Saved_Figs
        # clear_output()
        files.download(data_name + '_Saved_Figs.zip')

        '''Sort Cells'''
        cell_df = cell_sorting(abf_recordings_df)
        results['cell_df'] = cell_df

        '''Consolidate to Cells'''
        list_types = ['Recording_name','protocol','abf_timestamp', 'channelList']
        any_types = [] + dataset['file_naming_scheme']
        cell_df_con = cell_consolidation(cell_df,list_types,any_types)
        results['cell_df_con'] = cell_df_con

        '''Simplify IV Data'''
        cols_to_simplify = ['IV_Early', 'IV_Steady_State']
        cell_df_nd = simplify_dicts(cell_df_con,cols_to_simplify)
        results['cell_df_nd'] = cell_df_nd

        '''Make Excell Friendly'''
        keys_and_data_cols={'Stim_Levels_(pA)': ['Stim_Levels_(pA)', 'Spike_Counts' ],
                        'IV_Early_(V_stim)': ['IV_Early_(V_stim)', 'IV_Early_(I_peak)', 'IV_Steady_State_(I_mean)']}
        cell_df_csv = csv_frinedly(cell_df_nd,keys_and_data_cols)
        results['cell_df_csv'] = cell_df_csv


        ''' Convert to Current Density'''
        size_col = 'Cmq_160.0'
        current_col_list = ['IV_Early_(I_peak)_', 'IV_Steady_State_(I_mean)_']
        cell_df_csv = current_density_correction(cell_df_csv, size_col, current_col_list)
        results['cell_df_csv'] = cell_df_csv

        '''Abridge DataFrame'''
        abrg_exclusions = ['Recording_name',
                        'protocol', 'abf_timestamp', 'channelList',  'Ra_10.0', 'Rm_10.0', 'tau_10.0', 'Cmq_10.0', 'Cmf_10.0',
                        'Cmqf_10.0',  'Cmf_160.0', 'Cmqf_160.0', 'Cm_pc_160.0',
                        'Gain_R2', 'Stim_Levels_(pA)', 'Spike_Counts',  'Gain_Vh',  'Vhold_spike',
                            'Rin_Rsqr',  'Ramp_AP_thresh', 'Ramp_Vh', 'Ramp_Rheobase',
                        'v_half','is_compensated','sum_delta'
                        'IV_Early_(range)', 'IV_Early_(I_peak)', 'IV_Early_(I_mean)', 'IV_Early_(V_stim)', 'IV_Steady_State_(range)',
                        'IV_Steady_State_(I_peak)', 'IV_Steady_State_(I_mean)', 'IV_Steady_State_(V_stim)', ]

        abrg_keep = [c for c in cell_df_csv.columns if c not in abrg_exclusions]
        cell_df_csv_abrg = cell_df_csv[abrg_keep]
        results['cell_df_csv_abrg'] = cell_df_csv_abrg

        '''Stratify Cells By Type'''
        strat_df_dict = stratify_rec(cell_df_csv_abrg,strat_cols)
        strat_df_dict,_ = flatten_dict(strat_df_dict,{})
        write_strat_dfs(strat_df_dict, dataset['data_name']+'_results_stratified')
        results['strat_df_dict'] = strat_df_dict
    except: return results
    return results